In [262]:
import numpy as np 
from qiskit import *                 
from qiskit_aer import *             
from qiskit_ibm_runtime import *

In [263]:
#user_num = int(input("Enter the max for your QRNG range:"))                         # User input for max range
#num_qubits = int(np.ceil(np.log2(user_num)))                                        # Obtains number of qubits for the circuit

num_qubits = 100                 # Number of qubits to run on
chunk_size = 30                  # Size of the chunking for the mod2 and iteration methods
division_size = 10               # Size of the division for the concatenation method

In [264]:
QiskitRuntimeService.save_account(channel = 'ibm_quantum', token = '599b9581ddea8584dc2cc12731140ed8e9e90bee9abfc20d315ccbea501fbcfdc513b80379e3a99c0aedc6cdc4671364c669da201791f22a57c62cee9e03554a',
                                  overwrite = True, set_as_default = True)
service = QiskitRuntimeService(instance = "ibm-q/open/main")

In [265]:
def number_generator_simulator(num_qubits):
    circ = QuantumCircuit(num_qubits, num_qubits)                 # Creates circuit with number of qubits obtained
    circ.h(range(num_qubits))                                     # Applies a hadamard gate to all qubits
    circ.measure(range(num_qubits), range(num_qubits))            # Measures all qubits and assigns them to classical bits

    simulator = AerSimulator()                                         # Lets us use the Aer Simulator 
    compiled_circuit = transpile(circ, simulator)                      # Compiled circuit using Aer 
    result = simulator.run(compiled_circuit, shots = 1024).result()      # Result with the 10000 shots as to not run forever on IBM machines
    counts = result.get_counts()                                       # Assigns counts with the result
    
    return counts

In [266]:
def number_generator_brisbane(num_qubits):
    circ = QuantumCircuit(num_qubits, num_qubits)                 # Creates circuit with number of qubits obtained
    circ.h(range(num_qubits))                                     # Applies a hadamard gate to all qubits
    circ.measure(range(num_qubits), range(num_qubits))            # Measures all qubits and assigns them to classical bits

    brisbane_backend = service.backend('ibm_brisbane')            # Creates a backend with ibm_brisbane

    # Runs the QRNG on ibm_brisbane with 1024 shots
    pm = generate_preset_pass_manager(backend = brisbane_backend, optimization_level = 3)
    isa_circuit = pm.run(circ)
    with Session(backend = brisbane_backend) as session:
        sampler = Sampler()
        job = sampler.run([isa_circuit], shots = 1024)
        print(job.status())
        print(job.result())
        counts = job.result()[0].data.c.get_counts()
    
    return counts

In [267]:
def number_generator_sherbrooke(num_qubits):
    circ = QuantumCircuit(num_qubits, num_qubits)                 # Creates circuit with number of qubits obtained
    circ.h(range(num_qubits))                                     # Applies a hadamard gate to all qubits
    circ.measure(range(num_qubits), range(num_qubits))            # Measures all qubits and assigns them to classical bits

    sherbrooke_backend = service.backend('ibm_sherbrooke')        # Creates a backend with ibm_sherbrooke

    # Runs the QRNG on ibm_sherbrooke with 1024 shots
    pm = generate_preset_pass_manager(backend = sherbrooke_backend, optimization_level = 3)
    isa_circuit = pm.run(circ)
    with Session(backend = sherbrooke_backend) as session:
        sampler = Sampler()
        job = sampler.run([isa_circuit], shots = 1024)
        print(job.status())
        print(job.result())
        counts = job.result()[0].data.c.get_counts()
    
    return counts

In [268]:
# Mod 2 method functions
# Memory limitations (simulator only 30) -> "chunk" bit input
# chunk_size is # of qubits to divide chunk in to
def modChunker(chunk_size, machine):
    finalRN = ''

    if machine == 'ibm_brisbane':
        for i in range(0, int(num_qubits / chunk_size)):
            counts = number_generator_brisbane(chunk_size)
            rand_value = obtain_rand(counts)
            finalRN += rand_value
    elif machine == 'ibm_sherbrooke':
        for i in range(0, int(num_qubits / chunk_size)):
            counts = number_generator_sherbrooke(chunk_size)
            rand_value = obtain_rand(counts)
            finalRN += rand_value
    elif machine == 'simulator': 
        for i in range(0, int(num_qubits / chunk_size)):
            counts = number_generator_simulator(chunk_size)
            rand_value = obtain_rand(counts)
            finalRN += rand_value


    remainderSize = int(num_qubits - np.floor(num_qubits / chunk_size) * chunk_size)

    if remainderSize != 0:
        if machine == 'ibm_brisbane':
            counts = number_generator_brisbane(remainderSize)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value
        elif machine == 'ibm_sherbrookoe':
            counts = number_generator_sherbrooke(remainderSize)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value
        elif machine == 'simulator':
            counts = number_generator_simulator(remainderSize)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value

    return finalRN

# 
def obtain_ties(counts, machine):
    max_value = max(counts.values())
    max_keys = [key for key, value in counts.items() if value == max_value]
    max_dict = {key: counts[key] for key in max_keys}

    while (len(max_keys) > 1):
        max_keys = tie_breaker(max_keys, machine)

    return max_keys[0]

#  
def tie_breaker(ties, machine):
    new_num_qubits = int(np.ceil(np.log2(len(ties))))

    if machine == 'ibm_brisbane':
        counts = number_generator_brisbane(new_num_qubits)
    elif machine == 'ibm_sherbrooke':
        counts = number_generator_sherbrooke(new_num_qubits)
    elif machine == 'simulator':
        counts = number_generator_simulator(new_num_qubits)

    # Discard out of range
    counts = {key: value for key, value in counts.items() if int(key,2) < len(ties)}
    
    max_value = max(counts.values()) # Max among the ties
    max_keys = [key for key, value in counts.items() if value == max_value] # Keys (indexes!) that tied
    max_dict = {key: counts[key] for key in max_keys} # Dictionary of tied index->value's
    
    # Need to use new ties (indexes of original ties) to index which ties make it
    new_ties = []
    for key in max_keys:
        # Ensure that 'key' is treated correctly for indexing
        index = int(key, 2)  # Convert binary string key to integer
        new_ties.append(ties[index])  # Append the corresponding original tie

    return new_ties

# 
def obtain_rand(counts):
    # Get all values
    max_value = max(counts.values())
    max_keys = [key for key, value in counts.items() if value == max_value]
    max_dict = {key: counts[key] for key in max_keys}
    # Sum all Q-bit entries for tied keys
    # Get the first key
    new_key = [int(i) for i in max_keys[0]]
    for i in range(1, len(max_keys)):
        for j in range(0, len(new_key)):
            new_key[j] += int(max_keys[i][j])

    super_new_key = ''
    # Bitwise XOR (mod % 2) on each qubit
    for i in range(0, len(new_key)):
        # super_new_key[i] = str(int(new_key[i]) % 2)
        super_new_key += str(int(new_key[i]) % 2)

    return super_new_key

In [269]:
# Iteration method functions
# Memory limitations (simulator only 30) -> "chunk" bit input
# chunk_size is # of qubits to divide chunk in to
def iterationChunker(chunk_size, machine):
    finalRN = ''

    if machine == 'ibm_brisbane':
        for i in range(0, int(num_qubits / chunk_size)):
            counts = number_generator_brisbane(chunk_size)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value
    elif machine == 'ibm_sherbrooke':
        for i in range(0, int(num_qubits / chunk_size)):
            counts = number_generator_sherbrooke(chunk_size)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value
    elif machine == 'simulator':
        for i in range(0, int(num_qubits / chunk_size)):
            counts = number_generator_simulator(chunk_size)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value

    remainderSize = int(num_qubits - np.floor(num_qubits / chunk_size) * chunk_size)

    if remainderSize != 0:
        if machine == 'ibm_brisbane':
            counts = number_generator_brisbane(remainderSize)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value
        elif machine == 'ibm_sherbrookoe':
            counts = number_generator_sherbrooke(remainderSize)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value
        elif machine == 'simulator':
            counts = number_generator_simulator(remainderSize)
            rand_value = obtain_ties(counts, machine)
            finalRN += rand_value

    return finalRN

In [270]:
# Concatenation method functions
#
def concatenation(num_qubits, division_size, machine):
    extra = num_qubits % division_size
    iterations = int((num_qubits - extra) / division_size)
    final_output = []

    if machine == 'ibm_brisbane':
        for iteration in range(iterations):
            division_chunk = None

            while division_chunk is None:
                counts = number_generator_brisbane(division_size)
                division_chunk = check_tie(counts)
            final_output.append(division_chunk)

        if extra != 0:
            division_chunk = None

            while division_chunk is None:
                counts = number_generator_brisbane(extra)
                division_chunk = check_tie(counts)
            final_output.append(division_chunk)

        return ''.join(final_output)
    elif machine == 'ibm_sherbrooke':
        for iteration in range(iterations):
            division_chunk = None

            while division_chunk is None:
                counts = number_generator_sherbrooke(division_size)
                division_chunk = check_tie(counts)
            final_output.append(division_chunk)

        if extra != 0:
            division_chunk = None

            while division_chunk is None:
                counts = number_generator_sherbrooke(extra)
                division_chunk = check_tie(counts)
            final_output.append(division_chunk)

        return ''.join(final_output)
    elif machine == 'simulator':
        for iteration in range(iterations):
            division_chunk = None

            while division_chunk is None:
                counts = number_generator_simulator(division_size)
                division_chunk = check_tie(counts)
            final_output.append(division_chunk)

        if extra != 0:
            division_chunk = None

            while division_chunk is None:
                counts = number_generator_simulator(extra)
                division_chunk = check_tie(counts)
            final_output.append(division_chunk)

        return ''.join(final_output)

# Checks for any ties in the any of the divisions 
def check_tie(counts):
    max_value = max(counts.values())
    max_keys = [key for key, value in counts.items() if value == max_value]

    if len(max_keys) > 1:
        return None
    else:
        return max_keys[0]

In [271]:
# Mod 2 method utilization
#mod2_final_val_brisbane = modChunker(chunk_size, 'ibm_brisbane')
#mod2_final_val_sherbrooke = modChunker(chunk_size, 'ibm_sherbrooke')
#mod2_final_val_simulator = modChunker(chunk_size, 'simulator')

#print(f'Final random qubit state on ibm_brisbane:', mod2_final_val_brisbane)
#print(f'Final random qubit state on ibm_sherbrooke:', mod2_final_val_sherbrooke)
#print(f'Final random qubit state on AerSimulator:', mod2_final_val_simulator)

Final random qubit state on AerSimulator: 0010111101010000101101011001100111010001110000101110010110001001111011111101011011110100010001011000


In [272]:
# Iteration method utilization
#iteration_final_val_brisbane = iterationChunker(chunk_size, 'ibm_brisbane')
#iteration_final_val_sherbrooke = iterationChunker(chunk_size, 'ibm_sherbrooke')
#iteration_final_val_simulator = iterationChunker(chunk_size, 'simulator')

#print(f'Final random qubit state on ibm_brisbane:', iteration_final_val_brisbane)
#print(f'Final random qubit state on ibm_sherbrooke:', iteration_final_val_sherbrooke)
#print(f'Final random qubit state on AerSimulator:', iteration_final_val_simulator)

Final random qubit state on AerSimulator: 0001000000011001011111100000011000110101000100100000101111000001001010010110101100100010100101000101


In [273]:
# Concatenation method utilization
#concatenation_final_val_brisbane = concatenation(num_qubits, division_size, 'ibm_brisbane')
#concatenation_final_val_sherbrooke = concatenation(num_qubits, division_size, 'ibm_sherbrooke')
#concatenation_final_val_simulator = concatenation(num_qubits, division_size, 'simulator')

#print(f'Final random qubit state on ibm_brisbane:', concatenation_final_val_brisbane)
#print(f'Final random qubit state on ibm_sherbrooke:', concatenation_final_val_sherbrooke)
#print(f'Final random qubit state on AerSimulator:', concatenation_final_val_simulator)

Final random qubit state on AerSimulator: 1101001010010001100110110001010010010110001110010100001011101001000011001011100011111001010011011111


In [274]:
# File creation (machine_method_qubits_chunk/divisionSize)
# Mod 2 method file
#with open(f'brisbane_mod2_{num_qubits}_{chunk_size}.txt', 'w') as f:
#    f.write(mod2_final_val_brisbane)
#with open(f'sherbrooke_mod2_{num_qubits}_{chunk_size}.txt', 'w') as f:
#    f.write(mod2_final_val_sherbrooke)
#with open(f'simulator_mod2_{num_qubits}_{chunk_size}.txt', 'w') as f:
#    f.write(mod2_final_val_simulator)

# Iteration method file
#with open(f'brisbane_iteration_{num_qubits}_{chunk_size}.txt', 'w') as f:
#    f.write(iteration_final_val_brisbane)
#with open(f'sherbrooke_iteration_{num_qubits}_{chunk_size}.txt', 'w') as f:
#    f.write(iteration_final_val_sherbrooke)
#with open(f'simulator_iteration_{num_qubits}_{chunk_size}.txt', 'w') as f:
#    f.write(iteration_final_val_simulator)

# Concatenation method file
#with open(f'brisbane_concatenation_{num_qubits}_{division_size}.txt', 'w') as f:
#    f.write(concatenation_final_val_brisbane)
#with open(f'sherbrooke_concatenation_{num_qubits}_{division_size}.txt', 'w') as f:
#    f.write(concatenation_final_val_sherbrooke)
#with open(f'simulator_concatenation_{num_qubits}_{division_size}.txt', 'w') as f:
#    f.write(concatenation_final_val_simulator)